# Optimal POVM

Adapted from [QGopt/docs/optimal-povm](https://qgopt.readthedocs.io/en/latest/optimal_povm.html)

In [ ]:
import numpy as np
import torch

import numqi


In [ ]:
class OptimalPOVMModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.theta = torch.nn.Parameter(torch.rand(4,2,2,2, dtype=torch.float64))
        self.rho_list_T = []

    def forward(self):
        povm = self._get_povm()
        tmp0 = torch.linalg.inv(povm.reshape(4,4))
        T = torch.stack([tmp0[0].real, tmp0[3].real, tmp0[2].real, tmp0[2].imag])
        prob = torch.einsum(self.rho_list_T, [0,1], povm.reshape(4,4), [2,1], [0,2]).real
        tmp0 = torch.einsum(prob, [0,1], prob, [0,2], [0,1,2])
        tmp1 = (torch.diag_embed(prob, dim1=1, dim2=2) - tmp0).mean(axis=0)
        covariance = torch.einsum(T, [0,1], tmp1, [1,2], T, [3,2], [0,3])
        EVL = torch.linalg.eigvalsh(covariance)
        loss = torch.log(EVL[1:]).sum()
        return loss

    def set_rho_list(self, rho_list):
        assert (rho_list.ndim==3) and (rho_list.shape[1]==2) and (rho_list.shape[2]==2)
        self.rho_list_T = torch.tensor(rho_list.transpose(0,2,1).reshape(-1,4), dtype=torch.complex128)

    def _get_povm(self):
        tmp0 = torch.complex(self.theta[:,:,:,0], self.theta[:,:,:,1])
        tmp1 = numqi.param.matrix_to_kraus_op(tmp0)
        ret = torch.einsum(tmp1, [0,1,2], tmp1.conj(), [0,1,3], [0,2,3])
        return ret

    def get_povm(self):
        with torch.no_grad():
            ret = self._get_povm().numpy()
        return ret


Generate random density matrix from Bloch sphere.

In [ ]:
num_sample = 10000
tmp0 = numqi.random.rand_n_ball(3, size=num_sample).T.reshape(3,-1,1,1)/2
rho_list = numqi.gate.I/2 + numqi.gate.X*tmp0[0] + numqi.gate.Y*tmp0[1] + numqi.gate.Z*tmp0[2]


Build model for optimal POVM.

In [ ]:
model = OptimalPOVMModel()
model.set_rho_list(rho_list)
theta_optim = numqi.optimize.minimize(model, theta0='uniform', num_repeat=1, tol=1e-7)


Theoretically, the optimal POVM gives

$$ \mathrm{Tr}[M_iM_j]=\frac{2\delta_{ij}+1}{12} $$

In [ ]:
povm = model.get_povm()
cross = np.einsum(povm, [0,1,2], povm, [3,2,1], [0,3], optimize=True).real
print(f'Founded POVM: Tr[Mi Mj]=\n{np.around(cross, 5)}\n')

cross_analytical = 1/12 + 1/6 * np.eye(4)
print(f'Optimal POVM (theoretical): Tr[Mi Mj]=\n{np.around(cross_analytical, 5)}')
